In [1]:
import pandas as pd  # For handling data in DataFrame
import numpy as np
from selenium import webdriver  # For controlling the web browser and interacting with HTML content
from selenium.webdriver.chrome.service import Service  # For setting up ChromeDriver as a service
from selenium.webdriver.common.by import By  # To locate HTML elements by XPath
from selenium.webdriver.chrome.options import Options  # For handling Chrome options
from webdriver_manager.chrome import ChromeDriverManager  # For automatic ChromeDriver installation
import time  # For handling wait times
import random  # For randomizing scroll and wait times

In [3]:
#Kaggle Dataset to Dataframe
movies_df = pd.read_csv('movies.csv', encoding = 'utf-8')

In [5]:
display(movies_df)

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7663,More to Life,NaN,Drama,2020,"October 23, 2020 (United States)",3.1,18.0,Joseph Ebanks,Joseph Ebanks,Shannon Bond,United States,7000.0,NaN,NaN,90.0
7664,Dream Round,NaN,Comedy,2020,"February 7, 2020 (United States)",4.7,36.0,Dusty Dukatz,Lisa Huston,Michael Saquella,United States,NaN,NaN,Cactus Blue Entertainment,90.0
7665,Saving Mbango,NaN,Drama,2020,"April 27, 2020 (Cameroon)",5.7,29.0,Nkanya Nkwai,Lynno Lovert,Onyama Laura,United States,58750.0,NaN,Embi Productions,NaN
7666,It's Just Us,NaN,Drama,2020,"October 1, 2020 (United States)",NaN,NaN,James Randall,James Randall,Christina Roz,United States,15000.0,NaN,NaN,120.0


In [7]:
movies_df.isna().sum()

name           0
rating        77
genre          0
year           0
released       2
score          3
votes          3
director       0
writer         3
star           1
country        3
budget      2171
gross        189
company       17
runtime        4
dtype: int64

In [9]:
movies_df.dtypes

name         object
rating       object
genre        object
year          int64
released     object
score       float64
votes       float64
director     object
writer       object
star         object
country      object
budget      float64
gross       float64
company      object
runtime     float64
dtype: object

In [11]:
movies_df['rating'].unique()

array(['R', 'PG', 'G', nan, 'Not Rated', 'NC-17', 'Approved', 'TV-PG',
       'PG-13', 'Unrated', 'X', 'TV-MA', 'TV-14'], dtype=object)

Start on web scraping and crawling to determine what should be cleaned. IMDb Top 250 movies

In [13]:
# function to scroll from the top to the bottom of the web page
def random_scroll(browser, total_wait_time):
    # get the total height of the page
    total_height = browser.execute_script("return document.body.scrollHeight")
    
    # number of steps to scroll (you can adjust this number)
    scroll_steps = random.randint(3, 10) # randomize how many scroll steps we will use
    
    # calculate the height to scroll on each step
    scroll_increment = total_height // scroll_steps

    # calculate the total time available for scrolling each step
    time_per_step = total_wait_time / scroll_steps
    
    # random scrolling across time
    for step in range(scroll_steps):
        # scroll by the increment (dividing total height by number of steps)
        browser.execute_script(f"window.scrollBy(0, {scroll_increment});")
        
        # random wait time between scrolls to simulate varying speed
        random_wait = random.uniform(0.5 * time_per_step, 1.5 * time_per_step)  # randomize the wait within a range
        time.sleep(random_wait)
        
    # final scroll to make sure you are at the very bottom (in case it didn't exactly match)
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")


In [192]:
browser = webdriver.Chrome()
url = "https://www.imdb.com/chart/top/?ref_=nv_mv_250"
browser.get(url)
browser.maximize_window()

In [18]:
# find all <div> elements with the class 'ipc-metadata-list-summary-item'
title_divs = browser.find_elements(By.CLASS_NAME, "ipc-metadata-list-summary-item")

# extract the text from each <div> and store it in a list
movie_blocks = [div.text for div in title_divs]

print(movie_blocks)

['1. The Shawshank Redemption\n1994\n2h 22m\nR\n9.3\n (3M)\nRate\nMark as watched', '2. The Godfather\n1972\n2h 55m\nR\n9.2\n (2.1M)\nRate\nMark as watched', '3. The Dark Knight\n2008\n2h 32m\nPG-13\n9.0\n (3M)\nRate\nMark as watched', '4. The Godfather Part II\n1974\n3h 22m\nR\n9.0\n (1.4M)\nRate\nMark as watched', '5. 12 Angry Men\n1957\n1h 36m\nApproved\n9.0\n (920K)\nRate\nMark as watched', '6. The Lord of the Rings: The Return of the King\n2003\n3h 21m\nPG-13\n9.0\n (2.1M)\nRate\nMark as watched', "7. Schindler's List\n1993\n3h 15m\nR\n9.0\n (1.5M)\nRate\nMark as watched", '8. Pulp Fiction\n1994\n2h 34m\nR\n8.9\n (2.3M)\nRate\nMark as watched', '9. The Lord of the Rings: The Fellowship of the Ring\n2001\n2h 58m\nPG-13\n8.9\n (2.1M)\nRate\nMark as watched', '10. The Good, the Bad and the Ugly\n1966\n2h 58m\nR\n8.8\n (849K)\nRate\nMark as watched', '11. Forrest Gump\n1994\n2h 22m\nPG-13\n8.8\n (2.4M)\nRate\nMark as watched', '12. The Lord of the Rings: The Two Towers\n2002\n2h 59m\n

In [20]:
# create an empty list to store the extracted data
title_text = []

for div in title_divs:

    try: 
        title = div.find_element(By.CLASS_NAME, "ipc-title__text")
    except:
        title = ""
    
    title_text.append(title.text)

print(len(title_text))
print("-"*30)
print(title_text)

250
------------------------------
['1. The Shawshank Redemption', '2. The Godfather', '3. The Dark Knight', '4. The Godfather Part II', '5. 12 Angry Men', '6. The Lord of the Rings: The Return of the King', "7. Schindler's List", '8. Pulp Fiction', '9. The Lord of the Rings: The Fellowship of the Ring', '10. The Good, the Bad and the Ugly', '11. Forrest Gump', '12. The Lord of the Rings: The Two Towers', '13. Fight Club', '14. Inception', '15. Star Wars: Episode V - The Empire Strikes Back', '16. The Matrix', '17. Goodfellas', "18. One Flew Over the Cuckoo's Nest", '19. Interstellar', '20. Se7en', "21. It's a Wonderful Life", '22. The Silence of the Lambs', '23. Seven Samurai', '24. Saving Private Ryan', '25. City of God', '26. The Green Mile', '27. Life Is Beautiful', '28. Terminator 2: Judgment Day', '29. Star Wars: Episode IV - A New Hope', '30. Back to the Future', '31. Spirited Away', '32. The Pianist', '33. Gladiator', '34. Parasite', '35. Psycho', '36. The Lion King', '37. Grav

In [60]:
rankings = []
titles = []

for movie in title_text:
    # Split the string by the first period (.)
    rank, title = movie.split('.', 1)
    
    # Append the rank and title to their respective lists
    rankings.append(int(rank.strip()))
    titles.append(title.strip())

print("Rankings:", rankings)
print("Titles:", titles)

Rankings: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 22

In [62]:
IMDb_df = pd.DataFrame({
    'Rank': rankings,
    'Movie Title': titles
})

In [64]:
display(IMDb_df)

,Rank,Movie Title
0,1,The Shawshank Redemption
1,2,The Godfather
2,3,The Dark Knight
3,4,The Godfather Part II
4,5,12 Angry Men
...,...,...
245,246,Groundhog Day
246,247,The Help
247,248,Amores Perros
248,249,Rebecca


In [92]:
url_elements = browser.find_elements(By.XPATH, '//*[@id="__next"]/main/div/div[3]/section/div/div[2]/div/ul/li/div/div/div/div/div[2]/div[1]//a')

website_url = []

for u in url_elements:
    url = u.get_attribute('href')
    website_url.append(url)

website_url

['https://www.imdb.com/title/tt0111161/?ref_=chttp_t_1',
 'https://www.imdb.com/title/tt0068646/?ref_=chttp_t_2',
 'https://www.imdb.com/title/tt0468569/?ref_=chttp_t_3',
 'https://www.imdb.com/title/tt0071562/?ref_=chttp_t_4',
 'https://www.imdb.com/title/tt0050083/?ref_=chttp_t_5',
 'https://www.imdb.com/title/tt0167260/?ref_=chttp_t_6',
 'https://www.imdb.com/title/tt0108052/?ref_=chttp_t_7',
 'https://www.imdb.com/title/tt0110912/?ref_=chttp_t_8',
 'https://www.imdb.com/title/tt0120737/?ref_=chttp_t_9',
 'https://www.imdb.com/title/tt0060196/?ref_=chttp_t_10',
 'https://www.imdb.com/title/tt0109830/?ref_=chttp_t_11',
 'https://www.imdb.com/title/tt0167261/?ref_=chttp_t_12',
 'https://www.imdb.com/title/tt0137523/?ref_=chttp_t_13',
 'https://www.imdb.com/title/tt1375666/?ref_=chttp_t_14',
 'https://www.imdb.com/title/tt0080684/?ref_=chttp_t_15',
 'https://www.imdb.com/title/tt0133093/?ref_=chttp_t_16',
 'https://www.imdb.com/title/tt0099685/?ref_=chttp_t_17',
 'https://www.imdb.com/

In [104]:
awards = []
for url in website_url:
        browser.get(url)
        time.sleep(random.uniform(1, 3))
        award_elements = browser.find_elements(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[1]/div/ul/li/div/ul/li/span')
        for award in award_elements:
            awards.append(award.text)

display(awards)

['21 wins & 42 nominations total',
 '31 wins & 31 nominations total',
 '164 wins & 165 nominations total',
 '17 wins & 21 nominations total',
 '16 wins & 12 nominations total',
 '215 wins & 124 nominations total',
 '91 wins & 49 nominations total',
 '69 wins & 72 nominations total',
 '125 wins & 126 nominations total',
 '2 wins & 5 nominations total',
 '51 wins & 74 nominations total',
 '132 wins & 138 nominations total',
 '12 wins & 38 nominations total',
 '159 wins & 220 nominations total',
 '27 wins & 20 nominations total',
 '42 wins & 52 nominations total',
 '45 wins & 38 nominations total',
 '38 wins & 15 nominations total',
 '44 wins & 148 nominations total',
 '29 wins & 44 nominations total',
 '11 wins & 7 nominations total',
 '71 wins & 50 nominations total',
 '5 wins & 9 nominations total',
 '79 wins & 75 nominations total',
 '75 wins & 50 nominations total',
 '15 wins & 37 nominations total',
 '72 wins & 52 nominations total',
 '39 wins & 33 nominations total',
 '69 wins & 30

In [134]:
wins = []
nominations = []

for award in awards:
    if "wins" in award and "nominations" in award:
        parts = award.split(' & ')  # Split at " & "
        win_part = parts[0].split()[0]
        nomination_part = parts[1].split()[0]
    elif "wins" in award:  # Only wins, no nominations
        win_part = award.split()[0]  # Get the win count
        nomination_part = win_part  # Assign the same value to nominations
    elif "nominations" in award:  # Only nominations, no wins
        win_part = 0  # Assign 0 to wins
        nomination_part = award.split()[0]  # Get the nomination count
    else:
        win_part = 0
        nomination_part = 0
    
    wins.append(int(win_part))
    nominations.append(int(nomination_part))

In [136]:
print(len(wins))
print(len(nominations))

250
250


In [144]:
#Append onto IMDb_df
IMDb_df['Wins'] = wins
IMDb_df['Nominations'] = nominations

In [146]:
display(IMDb_df)

,Rank,Movie Title,Wins,Nominations
0,1,The Shawshank Redemption,21,42
1,2,The Godfather,31,31
2,3,The Dark Knight,164,165
3,4,The Godfather Part II,17,21
4,5,12 Angry Men,16,12
...,...,...,...,...
245,246,Groundhog Day,7,17
246,247,The Help,79,121
247,248,Amores Perros,55,24
248,249,Rebecca,10,10


In [198]:
#Get year for each movie so as not to use an incorrect movie with matching titles when merging
movie_divs = browser.find_elements(By.CLASS_NAME, 'ipc-metadata-list-summary-item')

years = []

for div in movie_divs:
    # Find all <span> elements inside this div
    span_elements = div.find_elements(By.TAG_NAME, 'span')
    
    if len(span_elements) > 1:
        # Extract the second span element (the year) and add it to the list
        year = span_elements[1].text.strip()
        years.append(year)

In [214]:
print(len(years))

250


In [216]:
#Append 'years' onto IMDb_df
IMDb_df['year'] = years

In [218]:
display(IMDb_df)

,Rank,Movie Title,Wins,Nominations,year
0,1,The Shawshank Redemption,21,42,1994
1,2,The Godfather,31,31,1972
2,3,The Dark Knight,164,165,2008
3,4,The Godfather Part II,17,21,1974
4,5,12 Angry Men,16,12,1957
...,...,...,...,...,...
245,246,Groundhog Day,7,17,1993
246,247,The Help,79,121,2011
247,248,Amores Perros,55,24,2000
248,249,Rebecca,10,10,1940


In [220]:
#Match column headers on both dataframes to properly merge
movies_df.rename(columns={'name': 'Movie Title'}, inplace=True)

In [222]:
display(movies_df)

,Movie Title,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7663,More to Life,NaN,Drama,2020,"October 23, 2020 (United States)",3.1,18.0,Joseph Ebanks,Joseph Ebanks,Shannon Bond,United States,7000.0,NaN,NaN,90.0
7664,Dream Round,NaN,Comedy,2020,"February 7, 2020 (United States)",4.7,36.0,Dusty Dukatz,Lisa Huston,Michael Saquella,United States,NaN,NaN,Cactus Blue Entertainment,90.0
7665,Saving Mbango,NaN,Drama,2020,"April 27, 2020 (Cameroon)",5.7,29.0,Nkanya Nkwai,Lynno Lovert,Onyama Laura,United States,58750.0,NaN,Embi Productions,NaN
7666,It's Just Us,NaN,Drama,2020,"October 1, 2020 (United States)",NaN,NaN,James Randall,James Randall,Christina Roz,United States,15000.0,NaN,NaN,120.0


In [230]:
movies_df.dtypes

Movie Title     object
rating          object
genre           object
year             int64
released        object
score          float64
votes          float64
director        object
writer          object
star            object
country         object
budget         float64
gross          float64
company         object
runtime        float64
dtype: object

In [232]:
IMDb_df.dtypes

Rank            int64
Movie Title    object
Wins            int64
Nominations     int64
year           object
dtype: object

In [234]:
#Convert 'year' to int64 to match movies_df
IMDb_df['year'] = IMDb_df['year'].astype('int')

In [236]:
inner_join = pd.merge(IMDb_df, movies_df, on=['Movie Title', 'year'], how='inner')

In [238]:
display(inner_join)

,Rank,Movie Title,Wins,Nominations,year,rating,genre,released,score,votes,director,writer,star,country,budget,gross,company,runtime
0,1,The Shawshank Redemption,21,42,1994,R,Drama,"October 14, 1994 (United States)",9.3,2400000.0,Frank Darabont,Stephen King,Tim Robbins,United States,25000000.0,2.881729e+07,Castle Rock Entertainment,142.0
1,3,The Dark Knight,164,165,2008,PG-13,Action,"July 18, 2008 (United States)",9.0,2400000.0,Christopher Nolan,Jonathan Nolan,Christian Bale,United States,185000000.0,1.005974e+09,Warner Bros.,152.0
2,6,The Lord of the Rings: The Return of the King,215,124,2003,PG-13,Action,"December 17, 2003 (United States)",8.9,1700000.0,Peter Jackson,J.R.R. Tolkien,Elijah Wood,New Zealand,94000000.0,1.146031e+09,New Line Cinema,201.0
3,7,Schindler's List,91,49,1993,R,Biography,"February 4, 1994 (United States)",8.9,1200000.0,Steven Spielberg,Thomas Keneally,Liam Neeson,United States,22000000.0,3.221612e+08,Universal Pictures,195.0
4,8,Pulp Fiction,69,72,1994,R,Crime,"October 14, 1994 (United States)",8.9,1900000.0,Quentin Tarantino,Quentin Tarantino,John Travolta,United States,8000000.0,2.139288e+08,Miramax,154.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,244,Into the Wild,13,6,2007,R,Adventure,"October 19, 2007 (United States)",8.1,588000.0,Sean Penn,Sean Penn,Emile Hirsch,United States,15000000.0,5.667590e+07,Paramount Vantage,148.0
138,246,Groundhog Day,7,17,1993,PG,Comedy,"February 12, 1993 (United States)",8.0,589000.0,Harold Ramis,Danny Rubin,Bill Murray,United States,14600000.0,7.107405e+07,Columbia Pictures,101.0
139,247,The Help,79,121,2011,PG-13,Drama,"August 10, 2011 (United States)",8.0,436000.0,Tate Taylor,Tate Taylor,Emma Stone,United States,25000000.0,2.166391e+08,DreamWorks,146.0
140,248,Amores Perros,55,24,2000,R,Drama,"April 13, 2001 (United States)",8.1,228000.0,Alejandro G. Iñárritu,Guillermo Arriaga,Emilio Echevarría,Mexico,2000000.0,2.090847e+07,Altavista Films,154.0


In [240]:
inner_join.isna().sum()

Rank            0
Movie Title     0
Wins            0
Nominations     0
year            0
rating          1
genre           0
released        0
score           0
votes           0
director        0
writer          0
star            0
country         0
budget         14
gross           1
company         0
runtime         0
dtype: int64

In [242]:
#Fixing Nan ratings
inner_join['rating'].unique()

array(['R', 'PG-13', 'PG', 'G', nan, 'TV-PG', 'Unrated', 'Not Rated'],
      dtype=object)

In [244]:
nan_ratings = inner_join[inner_join['rating'].isna()]
print(nan_ratings)

    Rank Movie Title  Wins  Nominations  year rating      genre  \
52    80    Das Boot    13           12  1981    NaN  Adventure   

                             released  score     votes           director  \
52  February 10, 1982 (United States)    8.3  237000.0  Wolfgang Petersen   

               writer             star       country  budget       gross  \
52  Wolfgang Petersen  Jürgen Prochnow  West Germany     NaN  11487676.0   

         company  runtime  
52  Bavaria Film    149.0  


In [246]:
inner_join.loc[inner_join['Movie Title'] == 'Das Boot', 'rating'] = 'R'

In [248]:
inner_join['rating'].isna().sum()

0

In [250]:
#Fixing Nan Budgets
nan_budgets = inner_join[inner_join['budget'].isna()]
print(nan_budgets)

     Rank                          Movie Title  Wins  Nominations  year  \
17     25                          City of God    75           50  2002   
31     43               Léon: The Professional     5           16  1994   
33     47                     The Intouchables    38           40  2011   
52     80                             Das Boot    13           12  1981   
54     82                    Princess Mononoke    14            6  1997   
55     83                           Your Name.    17           27  2016   
56     85                             3 Idiots    64           30  2009   
73    116                  Like Stars on Earth    28           18  2007   
75    121                               Snatch     4            7  2000   
78    125                             Downfall    22           34  2004   
79    126                               Dangal    41           38  2016   
99    166  Lock, Stock and Two Smoking Barrels    13            9  1998   
103   172                

In [252]:
#Imputing budgets that are sourceable via the internet
inner_join.loc[inner_join['Movie Title'] == 'City of God', 'budget'] = 3300000
inner_join.loc[inner_join['Movie Title'] == 'Léon: The Professional', 'budget'] = 16000000
inner_join.loc[inner_join['Movie Title'] == 'The Intouchables', 'budget'] = 13000000
inner_join.loc[inner_join['Movie Title'] == 'Das Boot', 'budget'] = 18500000
inner_join.loc[inner_join['Movie Title'] == 'Princess Mononoke', 'budget'] = 18000000
inner_join.loc[inner_join['Movie Title'] == 'Your Name.', 'budget'] = 7500000
inner_join.loc[inner_join['Movie Title'] == '3 Idiots', 'budget'] = 11375000
inner_join.loc[inner_join['Movie Title'] == 'Like Stars on Earth', 'budget'] = 2600000
inner_join.loc[inner_join['Movie Title'] == 'Snatch', 'budget'] = 10000000
inner_join.loc[inner_join['Movie Title'] == 'Downfall', 'budget'] = 16000000
inner_join.loc[inner_join['Movie Title'] == 'Dangal', 'budget'] = 10500000
inner_join.loc[inner_join['Movie Title'] == 'Lock, Stock and Two Smoking Barrels', 'budget'] = 1350000
inner_join.loc[inner_join['Movie Title'] == 'Trainspotting', 'budget'] = 3100000
inner_join.loc[inner_join['Movie Title'] == 'The Handmaiden', 'budget'] = 8800000

In [256]:
inner_join['budget'].isna().sum()

0

In [260]:
inner_join['gross'].isna().sum()

1

In [264]:
#Fixing Nan in 'gross' column
nan_gross = inner_join[inner_join['gross'].isna()]
print(nan_gross)

     Rank Movie Title  Wins  Nominations  year     rating  genre  \
141   250    Drishyam     2            8  2015  Not Rated  Crime   

                          released  score    votes         director  \
141  July 31, 2015 (United States)    8.2  75000.0  Nishikant Kamat   

            writer        star country     budget  gross           company  \
141  Jeethu Joseph  Ajay Devgn   India  9400000.0    NaN  Panorama Studios   

     runtime  
141    163.0  


In [266]:
inner_join.loc[inner_join['Movie Title'] == 'Drishyam', 'gross'] = 36000000

In [268]:
inner_join['gross'].isna().sum()

0

In [270]:
inner_join.isna().sum()

Rank           0
Movie Title    0
Wins           0
Nominations    0
year           0
rating         0
genre          0
released       0
score          0
votes          0
director       0
writer         0
star           0
country        0
budget         0
gross          0
company        0
runtime        0
dtype: int64

In [272]:
#dropping unnecessary columns
inner_join = inner_join.drop(columns=['star', 'writer', 'director', 'released'])

In [274]:
display(inner_join)

,Rank,Movie Title,Wins,Nominations,year,rating,genre,score,votes,country,budget,gross,company,runtime
0,1,The Shawshank Redemption,21,42,1994,R,Drama,9.3,2400000.0,United States,25000000.0,2.881729e+07,Castle Rock Entertainment,142.0
1,3,The Dark Knight,164,165,2008,PG-13,Action,9.0,2400000.0,United States,185000000.0,1.005974e+09,Warner Bros.,152.0
2,6,The Lord of the Rings: The Return of the King,215,124,2003,PG-13,Action,8.9,1700000.0,New Zealand,94000000.0,1.146031e+09,New Line Cinema,201.0
3,7,Schindler's List,91,49,1993,R,Biography,8.9,1200000.0,United States,22000000.0,3.221612e+08,Universal Pictures,195.0
4,8,Pulp Fiction,69,72,1994,R,Crime,8.9,1900000.0,United States,8000000.0,2.139288e+08,Miramax,154.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,244,Into the Wild,13,6,2007,R,Adventure,8.1,588000.0,United States,15000000.0,5.667590e+07,Paramount Vantage,148.0
138,246,Groundhog Day,7,17,1993,PG,Comedy,8.0,589000.0,United States,14600000.0,7.107405e+07,Columbia Pictures,101.0
139,247,The Help,79,121,2011,PG-13,Drama,8.0,436000.0,United States,25000000.0,2.166391e+08,DreamWorks,146.0
140,248,Amores Perros,55,24,2000,R,Drama,8.1,228000.0,Mexico,2000000.0,2.090847e+07,Altavista Films,154.0


In [288]:
#Getting rid of any scientific notation
pd.set_option('display.float_format', '{:,.1f}'.format)

In [290]:
display(inner_join)

,Rank,Movie Title,Wins,Nominations,year,rating,genre,score,votes,country,budget,gross,company,runtime
0,1,The Shawshank Redemption,21,42,1994,R,Drama,9.3,"2,400,000.0",United States,"25,000,000.0","28,817,291.0",Castle Rock Entertainment,142.0
1,3,The Dark Knight,164,165,2008,PG-13,Action,9.0,"2,400,000.0",United States,"185,000,000.0","1,005,973,645.0",Warner Bros.,152.0
2,6,The Lord of the Rings: The Return of the King,215,124,2003,PG-13,Action,8.9,"1,700,000.0",New Zealand,"94,000,000.0","1,146,030,912.0",New Line Cinema,201.0
3,7,Schindler's List,91,49,1993,R,Biography,8.9,"1,200,000.0",United States,"22,000,000.0","322,161,245.0",Universal Pictures,195.0
4,8,Pulp Fiction,69,72,1994,R,Crime,8.9,"1,900,000.0",United States,"8,000,000.0","213,928,762.0",Miramax,154.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,244,Into the Wild,13,6,2007,R,Adventure,8.1,"588,000.0",United States,"15,000,000.0","56,675,895.0",Paramount Vantage,148.0
138,246,Groundhog Day,7,17,1993,PG,Comedy,8.0,"589,000.0",United States,"14,600,000.0","71,074,049.0",Columbia Pictures,101.0
139,247,The Help,79,121,2011,PG-13,Drama,8.0,"436,000.0",United States,"25,000,000.0","216,639,112.0",DreamWorks,146.0
140,248,Amores Perros,55,24,2000,R,Drama,8.1,"228,000.0",Mexico,"2,000,000.0","20,908,467.0",Altavista Films,154.0


In [292]:
inner_join.dtypes

Rank             int64
Movie Title     object
Wins             int64
Nominations      int64
year             int32
rating          object
genre           object
score          float64
votes          float64
country         object
budget         float64
gross          float64
company         object
runtime        float64
dtype: object

In [294]:
inner_join.isna().sum()

Rank           0
Movie Title    0
Wins           0
Nominations    0
year           0
rating         0
genre          0
score          0
votes          0
country        0
budget         0
gross          0
company        0
runtime        0
dtype: int64

In [296]:
#Save merged dataframe to csv
inner_join.to_csv("juroe_project_proposal.csv", encoding = 'utf-8', index = False)